In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
tf.__version__

'2.12.0'

In [4]:
dataset,info = tfds.load('imdb_reviews/subwords8k',
                         with_info=True,
                         as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteOITLTF/imdb_reviews-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteOITLTF/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteOITLTF/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [5]:
tD,vD = dataset['train'],dataset['test']

In [6]:
encoder = info.features['text'].encoder

In [7]:
s = 'Hello, how are you'
es = encoder.encode(s)

In [8]:
es #encoding string

[4025, 8040, 2, 113, 29, 151]

In [9]:
encoder.vocab_size #total words in vocab

8185

In [10]:
for idx in es:
    print('{}-->{}'.format(idx,encoder.decode([idx])))

4025-->Hell
8040-->o
2-->, 
113-->how 
29-->are 
151-->you


In [11]:
tD = tD.shuffle(1000) #randomly pick 1000 training data
tD = tD.padded_batch(50) #create batches of 50
vD = vD.padded_batch(50) #create batches of 50

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size,10), #Feature extraction through encoding 
    tf.keras.layers.LSTM(12), #use LSTM in RNN, 12 units in LSTM
    tf.keras.layers.Dense(5), #hidden layer with 5 neurons
    tf.keras.layers.Dense(1), #1 unit or neuron , softmax layer as binary classification problem
])

In [13]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer= tf.keras.optimizers.Adam(1e-3),
              metrics= ['accuracy'])

In [14]:
history = model.fit(tD,epochs=2,
                    validation_data=vD,
                    validation_steps=30)

Epoch 1/2
500/500 [==============================] - 231s 458ms/step - loss: 0.6934 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.4993
Epoch 2/2
500/500 [==============================] - 226s 453ms/step - loss: 0.6930 - accuracy: 0.5003 - val_loss: 0.6935 - val_accuracy: 0.4993


In [15]:
vL,vA = model.evaluate(vD)

500/500 [==============================] - 37s 74ms/step - loss: 0.6929 - accuracy: 0.5004


In [16]:
print(vL,vA)

0.6929206848144531 0.5003600120544434
